# LLM for coding

In this mini tutorial we will demonstrate how LLM agents can be used effectively for coding.

We will use the previous notebook "0_code_testing" to see if with a few prompts LLM can reproduce our code and write tests for it 

In [2]:
# Data handling packages
import numpy as np  
import pandas as pd 
import pynwb  

# Plotting libraries
import matplotlib.pyplot as plt  

# Set the data root according to OS
import platform
from pathlib import Path

# Pandas display settings
pd.set_option('display.max_columns', None)  # Ensures all columns are shown when printing DataFrames

# Inline plotting for Jupyter Notebooks
%matplotlib inline  

ModuleNotFoundError: No module named 'pandas'

Let us consider loading in data from the Visual Behavior Neuropixels dataset we saw in a previous day. Specifically, let us load a single experimental session.

In [ ]:
platstring = platform.platform()

if 'Darwin' in platstring:
    # macOS 
    data_root = Path("/Volumes/Brain2025/")
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = Path("E:/")
elif ('amzn' in platstring):
    # then on CodeOcean
    data_root = Path("/data/")
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = Path("/media/$USERNAME/Brain2025/")

# pick a session_id and get session data
example_sessions = [1139846596, 1152811536, 1069461581 ]
this_session = str(example_sessions[0])
this_filename = f'ecephys_session_{this_session}.nwb'
nwb_path = Path('visual-behavior-neuropixels', 'behavior_ecephys_sessions', this_session, this_filename)

# access the session data with pynwb
session = pynwb.NWBHDF5IO(data_root / nwb_path).read()


Let us load in the trial data and metadata about each recorded unit.

In [ ]:
trials = session.trials.to_dataframe() 
units_table = session.units.to_dataframe()
electrodes_table = session.electrodes.to_dataframe()
units_table = units_table.join(electrodes_table, on = 'peak_channel_id')

In this metadata, we have important pieces of information such as the amplitude cutoff, inter-spike-interval (ISI) violations ratio, presence ratio, and activity drift of each recorded unit. Generally, we want to filter neurons by these quantities to find "good" neurons. For example, consider defining the following thresholds on these quantities below.

In [ ]:
max_amplitude_cutoff = 0.1
max_isi_violations_ratio = 0.5
min_presence_ratio = 0.7

When loading in the trial data for a specific neuron, we can check these quantities with if and print statements to make sure that our criteria are satisfied.

In [ ]:
id_no = 0
unit_amplitude_cutoff = units_table.iloc[id_no]['amplitude_cutoff']
unit_isi_violations_ratio = units_table.iloc[id_no]['isi_violations']
unit_presence_ratio = units_table.iloc[id_no]['presence_ratio']

if not unit_amplitude_cutoff <= max_amplitude_cutoff:
    print(f'Unit amplitude cutoff is {unit_amplitude_cutoff}, must be <= {max_amplitude_cutoff}')
if not unit_isi_violations_ratio <= max_isi_violations_ratio:
    print(f'ISI Violations ratio is {unit_isi_violations_ratio}, must be <= {max_isi_violations_ratio}')
if not unit_presence_ratio >= min_presence_ratio:
    print(f'Presence ratio is {unit_presence_ratio}, must be >= {min_presence_ratio}')

Alternatively, we can use assert statements instead.

In [ ]:
id_no = 0
unit_amplitude_cutoff = units_table.iloc[id_no]['amplitude_cutoff']
unit_isi_violations_ratio = units_table.iloc[id_no]['isi_violations']
unit_presence_ratio = units_table.iloc[id_no]['presence_ratio']

assert unit_amplitude_cutoff <= max_amplitude_cutoff, f'Unit amplitude cutoff is {unit_amplitude_cutoff}, must be <= {max_amplitude_cutoff}'
assert unit_isi_violations_ratio <= max_isi_violations_ratio, f'ISI Violations ratio is {unit_isi_violations_ratio}, must be <= {max_isi_violations_ratio}'
assert unit_presence_ratio >= min_presence_ratio, f'Presence ratio is {unit_presence_ratio}, must be >= {min_presence_ratio}'


### Let us try to reproduce some parts using ChatGPT UI

Try the following prompt:

```"Given a session, load the metadata—specifically, the trials and units—into separate DataFrames."```

Before accepting the result, evaluate what went wrong. Did we give too little information? 

Try this prompt:

```"You are a neuroscientist helping me analyze data from a recent experiment. The data is organized into multiple sessions, each of which can be loaded using: `session = pynwb.NWBHDF5IO(nwb_path).read()`. From each session, I need to extract metadata—specifically, trials and units—and convert them into separate pandas DataFrames. Write a Python script to accomplish this."```

Notice how much more specific this prompt is—you can refine it even further. Try the following:

```"You are a neuroscientist helping me analyze data from a recent experiment. The data is organized into multiple sessions, each of which can be loaded with: `session = pynwb.NWBHDF5IO(nwb_path).read()`. I need to extract metadata from a session—specifically, trials and units—and convert them into separate pandas DataFrames. Assume the session is already loaded. Write a concise Python script that performs this task. Do not save the DataFrames to CSV. Return only the script, with no additional explanation."```

Discuss: Can you think of a shorter prompt to produce the same or similar result? 


### If you are using Copilot

Let’s begin with a simple example. Following the logic from the previous notebook, our goal is to extract session metadata and organize it into DataFrames.

Try the following prompt:

```"Given a session, load the metadata—specifically, the trials and units—into separate DataFrames."```

Discuss: Why does this prompt work well with GitHub Copilot but not as effectively in the ChatGPT UI?

With Copilot, short and specific code-related instructions often lead to immediate and relevant completions, since it's optimized for in-context code generation. Let's test this difference by attempting a practical example: generating a function and then writing a test for it.


 
We’ll start by providing only the function description and see if Copilot can infer and generate the correct implementation. Try the following prompt:

```"Generate a function called `make_psth` with the following behavior: The function should compute a Peri-Stimulus Time Histogram (PSTH).```

```Parameters:```
   - spike_times: array-like, timestamps of all spikes (in seconds)
   - stim_times: array-like, timestamps of stimulus onsets (in seconds)
   - pre_window: float, time before stimulus to include in the PSTH (seconds)
   - post_window: float, time after stimulus to include in the PSTH (seconds)
   - bin_size: float, width of each time bin (seconds)

```Returns:```
   - firing_rates: 2D NumPy array of firing rates (shape: trials × bins)
   - bin_centers: 1D NumPy array of bin center times (relative to stimulus onset)"

How can we verify that this function works as intended?
One way is to write a test to ensure that make_psth executes properly and returns meaningful output. Try the following prompt:

```"Generate a test to ensure the function `make_psth` runs correctly and returns meaningful results."```

How do you know the test result is actually meaningful? Compare the result with a person sitting next to you. You should _always_ go back and verify the output yourself—never assume it's correct just because it runs.

Alternatively, for smaller code blocks or quick autocompletions, you can use Copilot to assist with implementation.

Exercise. Rewrite the function above using autocomplete. Make sure to check every line the Copilot generates for you. 